# Device discovery in the Linux /sys directory
* [man file formats and filesystems(5)](https://manpages.debian.org/bullseye/manpages/intro.5.en.html)
* [man sysfs.5](https://manpages.debian.org/bullseye/manpages/sysfs.5.en.html)
  * The files under sysfs provide information about devices, kernel modules, filesystems, and other kernel components.
  * The sysfs filesystem first appeared in Linux 2.6.0.
* [man proc filesystem](https://manpages.debian.org/bullseye/manpages/proc.5.en.html)
* [man udev](https://manpages.debian.org/bullseye/udev/udev.7.en.html)

In [2]:
include(joinpath(@__DIR__, "example_data.jl"))

TableDirectories.discriminant(data_frame, all_column_names) = 24×5 DataFrame
 Row │ partition  ro         size         start        stat
     │ Union…     SubStrin…  SubStrin…    Union…       SubStrin…
─────┼───────────────────────────────────────────────────────────────────────────────────
   1 │            0          2097151                         10        0        4      …
   2 │            1          203648                         573        0     7100     1…
   3 │            1          445384                         190        0     3286     1…
   4 │            0          0                               11        0       28      …
   5 │            0          0                                0        0        0      …
   6 │            0          0                                0        0        0      …
   7 │            0          0                                0        0        0      …
   8 │            0          0                                0        0        0    

                                                                                                                                                                                                                                                                                                                                                     /sys/devices/LNXSYSTM:00/LNXSYBU…  usb:v0BDAp0129d3960dcFFdscFFdpFF…  0                          enabled                suspended             DEVTYPE=usb_interface\nDRIVER=rt…
  17 │  1                 07                 00                 04                  01                  02                 /sys/bus/usb/drivers/usbfs                                                                                                                                                                                                                                                                                                                                              

,bAlternateSetting,bInterfaceClass,bInterfaceNumber,bInterfaceProtocol,bInterfaceSubClass
,SubStrin…,SubStrin…,SubStrin…,SubStrin…,SubStrin…
1,0,09,00,00,00
2,0,e0,00,01,01
3,0,e0,01,01,01
4,0,09,00,00,00
5,0,09,00,00,00
6,0,09,00,00,00
7,0,03,00,02,01
8,0,01,00,00,01
9,0,01,01,00,02


In [36]:
endpoint_numbering = sort(map(glob(glob"*/ep*/bEndpointAddress", "/sys/bus/usb/devices/")) do path
    filename, endpoint = splitpath(path)[6:7]
    device = split(filename, ":")
    (device, endpoint[4:5], filename)
end)
#endpoint_numbering[2]

39-element Vector{Tuple{Vector{SubString{String}}, String, String}}:
 (["1-0", "1.0"], "81", "1-0:1.0")
 (["1-1"], "00", "1-1")
 (["1-1", "1.0"], "81", "1-1:1.0")
 (["1-14"], "00", "1-14")
 (["1-14", "1.0"], "02", "1-14:1.0")
 (["1-14", "1.0"], "81", "1-14:1.0")
 (["1-14", "1.0"], "82", "1-14:1.0")
 (["1-14", "1.1"], "03", "1-14:1.1")
 (["1-14", "1.1"], "83", "1-14:1.1")
 (["1-2"], "00", "1-2")
 ⋮
 (["1-9", "1.0"], "01", "1-9:1.0")
 (["1-9", "1.0"], "82", "1-9:1.0")
 (["1-9", "1.1"], "03", "1-9:1.1")
 (["1-9", "1.1"], "84", "1-9:1.1")
 (["1-9", "1.2"], "05", "1-9:1.2")
 (["1-9", "1.2"], "86", "1-9:1.2")
 (["2-0", "1.0"], "81", "2-0:1.0")
 (["usb1"], "00", "usb1")
 (["usb2"], "00", "usb2")

In [67]:
using Test
ports = map(readlines(`find /sys/devices/pci0000\:00/0000\:00\:14.0/ -name port -ls`)) do line
    parts = split(line[110:end], "/")
    bus, insertion = split(parts[2], "-")
    @test bus ==  parts[1][4:4]
    #@test bus == parts[4][1:1]
    @test bus == parts[2][1:1]
    parts
end

11-element Vector{Vector{SubString{String}}}:
 ["usb1", "1-9", "port -> ..", "1-0:1.0", "usb1-port9"]
 ["usb1", "1-14", "port -> ..", "1-0:1.0", "usb1-port14"]
 ["usb1", "1-7", "port -> ..", "1-0:1.0", "usb1-port7"]
 ["usb1", "1-1", "port -> ..", "1-0:1.0", "usb1-port1"]
 ["usb1", "1-8", "port -> ..", "1-0:1.0", "usb1-port8"]
 ["usb1", "1-6", "port -> ..", "1-0:1.0", "usb1-port6"]
 ["usb1", "1-6", "1-6.2", "port -> ..", "1-6:1.0", "1-6-port2"]
 ["usb1", "1-6", "1-6.3", "port -> ..", "1-6:1.0", "1-6-port3"]
 ["usb1", "1-6", "1-6.1", "port -> ..", "1-6:1.0", "1-6-port1"]
 ["usb1", "1-2", "1-2.1", "port -> ..", "1-2:1.0", "1-2-port1"]
 ["usb1", "1-2", "port -> ..", "1-0:1.0", "usb1-port2"]

In [25]:
pairs = map(names(endpoint_data_frame)) do column_name
    (TableDirectories.column_frequency_stats(endpoint_data_frame, column_name)[:number_distinct], column_name)
end
filter(sort(pairs)) do pair
    pair[2][1:3] !== "ep_"
end

284-element Vector{Tuple{Int64, String}}:
 (1, "authorized")
 (1, "power/async")
 (1, "power/runtime_usage")
 (1, "subsystem")
 (1, "supports_autosuspend")
 (2, "bAlternateSetting")
 (2, "power/runtime_active_kids")
 (2, "power/runtime_enabled")
 (2, "usb1-port1/connect_type")
 (2, "usb1-port1/device")
 ⋮
 (4, "bInterfaceNumber")
 (4, "bNumEndpoints")
 (5, "bInterfaceSubClass")
 (6, "bInterfaceProtocol")
 (7, "bInterfaceClass")
 (7, "driver")
 (9, "firmware_node")
 (18, "modalias")
 (18, "uevent")

In [24]:
unique(endpoint_data_frame[!, ["driver", "bInterfaceClass", "bInterfaceProtocol"]])

,driver,bInterfaceClass,bInterfaceProtocol
,Union…,SubStrin…,SubStrin…
1,/sys/bus/usb/drivers/hub,09,00
2,,e0,01
3,/sys/bus/usb/drivers/usbhid,03,02
4,/sys/bus/usb/drivers/snd-usb-audio,01,00
5,/sys/bus/usb/drivers/usbhid,03,00
6,/sys/bus/usb/drivers/usbhid,03,01
7,/sys/bus/usb/drivers/usb-storage,08,50
8,/sys/bus/usb/drivers/rtsx_usb,ff,50
9,/sys/bus/usb/drivers/usbfs,07,04


In [22]:
endpoint_data_frame[!, ["driver", "bInterfaceClass"]] # "uevent", , "manufacturer", "product"

ArgumentError: ArgumentError: cell name :product not found in the data frame

In [4]:
using Glob
devs = map(glob(glob"*/*", "/sys/dev")) do path
    (path, readlink(path))
end

210-element Vector{Tuple{String, String}}:
 ("/sys/dev/block/11:0", "../../devices/pci0000:00/0000:00:17.0/ata3/host2/target2:0:0/2:0:0:0/block/sr0")
 ("/sys/dev/block/7:0", "../../devices/virtual/block/loop0")
 ("/sys/dev/block/7:1", "../../devices/virtual/block/loop1")
 ("/sys/dev/block/7:2", "../../devices/virtual/block/loop2")
 ("/sys/dev/block/7:3", "../../devices/virtual/block/loop3")
 ("/sys/dev/block/7:4", "../../devices/virtual/block/loop4")
 ("/sys/dev/block/7:5", "../../devices/virtual/block/loop5")
 ("/sys/dev/block/7:6", "../../devices/virtual/block/loop6")
 ("/sys/dev/block/7:7", "../../devices/virtual/block/loop7")
 ("/sys/dev/block/8:0", "../../devices/pci0000:00/0000:00:17.0/ata1/host0/target0:0:0/0:0:0:0/block/sda")
 ⋮
 ("/sys/dev/char/7:5", "../../devices/virtual/vc/vcs5")
 ("/sys/dev/char/7:6", "../../devices/virtual/vc/vcs6")
 ("/sys/dev/char/7:64", "../../devices/virtual/vc/vcsu")
 ("/sys/dev/char/7:65", "../../devices/virtual/vc/vcsu1")
 ("/sys/dev/char/7:66", ".

# Subsystems
* how to avoid uplinks?

In [5]:

using Glob
rows = map(union(glob(glob"*/subsystem", "/sys/devices"), union(glob(glob"*/*/subsystem", "/sys/devices")), union(glob(glob"*/*/*/subsystem", "/sys/devices")), union(glob(glob"*/*/*/*/subsystem", "/sys/devices")))) do path
    (path, readlink(path))
end
find_all("/sys/devices", "subsystem")

UndefVarError: UndefVarError: find_all not defined

In [6]:
subsystems = map(readlines(`find /sys -name subsystem -ls`)) do line
    link = line[60:end]
    split(link, " -> ")
end

ProcessFailedException: failed process: Process(`find /sys -name subsystem -ls`, ProcessExited(1)) [1]
